In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
Customers = pd.read_csv("Customers.csv")
Products = pd.read_csv("Products.csv")
Transactions = pd.read_csv("Transactions.csv")

In [5]:
merged_df = Transactions.merge(Products, on='ProductID').merge(Customers, on='CustomerID')

In [6]:
merged_df.shape

(1000, 13)

In [7]:
merged_df.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,ProductName,Category,Price_y,CustomerName,Region,SignupDate
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Andrea Jenkins,Europe,2022-12-03
1,T00761,C0199,P022,2024-10-01 05:57:09,4,550.16,137.54,HomeSense Wall Art,Home Decor,137.54,Andrea Jenkins,Europe,2022-12-03
2,T00626,C0199,P079,2024-08-17 12:06:08,2,834.74,417.37,ActiveWear Rug,Home Decor,417.37,Andrea Jenkins,Europe,2022-12-03
3,T00963,C0199,P008,2024-10-26 00:01:58,2,293.70,146.85,BookWorld Bluetooth Speaker,Electronics,146.85,Andrea Jenkins,Europe,2022-12-03
4,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Brittany Harvey,Asia,2024-09-04


In [8]:
# Account Age (Days)
merged_df['SignupDate'] = pd.to_datetime(merged_df['SignupDate'])
merged_df['AccountAge'] = (pd.to_datetime('today') - merged_df['SignupDate']).dt.days

In [9]:
# Total Spend per Customer
customer_total_spend = merged_df.groupby('CustomerID')['TotalValue'].sum().reset_index()
customer_total_spend.rename(columns={'TotalValue': 'TotalSpend'}, inplace=True)

In [10]:
# Total Transaction Count per Customer
customer_transaction_count = merged_df.groupby('CustomerID')['TransactionID'].count().reset_index()
customer_transaction_count.rename(columns={'TransactionID': 'TransactionCount'}, inplace=True)


In [11]:
# Average Spend per Transaction
customer_avg_spend = merged_df.groupby('CustomerID')['TotalValue'].mean().reset_index()
customer_avg_spend.rename(columns={'TotalValue': 'AvgSpend'}, inplace=True)

In [12]:
# Merge these features back into the main dataframe
merged_df = merged_df.merge(customer_total_spend, on='CustomerID', how='left')
merged_df = merged_df.merge(customer_transaction_count, on='CustomerID', how='left')
merged_df = merged_df.merge(customer_avg_spend, on='CustomerID', how='left')

In [13]:
# Frequency of Purchases (per day)
merged_df['Frequency'] = merged_df['TransactionCount'] / merged_df['AccountAge']

In [14]:
# Average Quantity per Transaction
customer_avg_quantity = merged_df.groupby('CustomerID')['Quantity'].mean().reset_index()
customer_avg_quantity.rename(columns={'Quantity': 'AvgQuantity'}, inplace=True)
merged_df = merged_df.merge(customer_avg_quantity, on='CustomerID', how='left')

In [15]:
# Recency of Last Purchase (in days)
last_purchase_date = merged_df.groupby('CustomerID')['TransactionDate'].max().reset_index()
last_purchase_date['LastPurchaseRecency'] = (pd.to_datetime('today') - pd.to_datetime(last_purchase_date['TransactionDate'])).dt.days
merged_df = merged_df.merge(last_purchase_date[['CustomerID', 'LastPurchaseRecency']], on='CustomerID', how='left')

In [16]:
# Merge region-specific features if needed (average spend per region, etc.)
region_avg_spend = merged_df.groupby('Region')['TotalValue'].mean().reset_index()
region_avg_spend.rename(columns={'TotalValue': 'RegionAvgSpend'}, inplace=True)
merged_df = merged_df.merge(region_avg_spend, on='Region', how='left')

In [17]:
category_spend = merged_df.groupby(['CustomerID', 'Category'])['TotalValue'].sum().reset_index()
category_spend_pivot = category_spend.pivot(index='CustomerID', columns='Category', values='TotalValue').fillna(0)
merged_df = pd.merge(merged_df, category_spend_pivot, on='CustomerID', how='left')

In [18]:
merged_df.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,ProductName,Category,Price_y,...,TransactionCount,AvgSpend,Frequency,AvgQuantity,LastPurchaseRecency,RegionAvgSpend,Books,Clothing,Electronics,Home Decor
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,...,4,494.82,0.005109,2.25,90,710.489872,0.00,0.0,594.38,1384.90
1,T00761,C0199,P022,2024-10-01 05:57:09,4,550.16,137.54,HomeSense Wall Art,Home Decor,137.54,...,4,494.82,0.005109,2.25,90,710.489872,0.00,0.0,594.38,1384.90
2,T00626,C0199,P079,2024-08-17 12:06:08,2,834.74,417.37,ActiveWear Rug,Home Decor,417.37,...,4,494.82,0.005109,2.25,90,710.489872,0.00,0.0,594.38,1384.90
3,T00963,C0199,P008,2024-10-26 00:01:58,2,293.70,146.85,BookWorld Bluetooth Speaker,Electronics,146.85,...,4,494.82,0.005109,2.25,90,710.489872,0.00,0.0,594.38,1384.90
4,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,...,4,642.70,0.028169,2.00,36,697.591606,1011.66,0.0,734.32,824.82


In [19]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   TransactionID        1000 non-null   object        
 1   CustomerID           1000 non-null   object        
 2   ProductID            1000 non-null   object        
 3   TransactionDate      1000 non-null   object        
 4   Quantity             1000 non-null   int64         
 5   TotalValue           1000 non-null   float64       
 6   Price_x              1000 non-null   float64       
 7   ProductName          1000 non-null   object        
 8   Category             1000 non-null   object        
 9   Price_y              1000 non-null   float64       
 10  CustomerName         1000 non-null   object        
 11  Region               1000 non-null   object        
 12  SignupDate           1000 non-null   datetime64[ns]
 13  AccountAge           1000 non-null

In [20]:
customer_data = merged_df[['CustomerID','AccountAge', 'TotalSpend', 'TransactionCount', 'AvgSpend', 'Frequency', 
                      'AvgQuantity', 'LastPurchaseRecency', 'RegionAvgSpend','Books', 
                       'Clothing','Electronics','Home Decor']]

In [21]:
customer_data = customer_data.drop_duplicates().reset_index()

In [22]:
customer_data['CustomerID'] = customer_data['CustomerID'].str.replace(r'\D', '', regex=True).astype(int)


In [23]:
customer_data.head()

,index,CustomerID,AccountAge,TotalSpend,TransactionCount,AvgSpend,Frequency,AvgQuantity,LastPurchaseRecency,RegionAvgSpend,Books,Clothing,Electronics,Home Decor
0,0,199,783,1979.28,4,494.820000,0.005109,2.250000,90,710.489872,0.00,0.0,594.38,1384.90
1,4,146,142,2570.80,4,642.700000,0.028169,2.000000,36,697.591606,1011.66,0.0,734.32,824.82
2,8,127,295,3232.88,6,538.813333,0.020339,1.833333,126,710.489872,488.63,0.0,1464.02,1280.23
3,14,87,288,6604.23,7,943.461429,0.024306,3.142857,132,721.554474,2399.91,909.6,3294.72,0.00
4,21,70,1046,3125.49,4,781.372500,0.003824,3.000000,109,710.489872,2223.45,0.0,902.04,0.00


In [24]:
customer_data.shape

(199, 14)

In [26]:
from sklearn.preprocessing import StandardScaler

# Select numerical features for scaling
features = ['AccountAge', 'TotalSpend', 'TransactionCount', 'AvgSpend', 'Frequency', 
                      'AvgQuantity', 'LastPurchaseRecency', 'RegionAvgSpend','Books', 
                       'Clothing','Electronics','Home Decor']  # Add other categories as needed

# Standardize the numerical features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_data[features])

In [27]:
scaled_features.shape

(199, 12)

In [28]:
cust = customer_data['CustomerID'].tolist()

In [31]:
similarity_matrix = cosine_similarity(scaled_features)

In [32]:
# Debugging: print the full similarity matrix
print("Similarity Matrix:")
print(np.round(similarity_matrix, 2))

Similarity Matrix:
[[ 1.    0.23  0.42 ...  0.32  0.11  0.06]
 [ 0.23  1.    0.55 ...  0.31  0.69 -0.18]
 [ 0.42  0.55  1.   ... -0.1   0.04 -0.09]
 ...
 [ 0.32  0.31 -0.1  ...  1.    0.35  0.39]
 [ 0.11  0.69  0.04 ...  0.35  1.    0.21]
 [ 0.06 -0.18 -0.09 ...  0.39  0.21  1.  ]]


In [33]:
def get_top_similar(customers, similarity_matrix, top_n=3):
    lookalike_dict = {}
    
    for idx, customer_id in enumerate(customers):
        # Extract similarity scores and exclude self-similarity (i != idx)
        customer_scores = [(i, score) for i, score in enumerate(similarity_matrix[idx]) if i != idx]
        
        # Sort by similarity score in descending order and get top N
        top_similar = sorted(customer_scores, key=lambda x: x[1], reverse=True)[:top_n]
        
        # Remove duplicates from top_similar (if any)
        top_similar = list(set(top_similar))  # Removes duplicate entries
        
        # Map indices back to Customer IDs
        lookalike_dict[customer_id] = [(customers[sim[0]], sim[1]) for sim in top_similar]
    
    return lookalike_dict


In [34]:
lookalike_dict = get_top_similar(cust, similarity_matrix, top_n=3)

In [35]:
# Print results for validation
print("\nLookalike Results:")
for customer, lookalikes in lookalike_dict.items():
    print(f"CustomerID: {customer}, Lookalikes: {lookalikes}")


Lookalike Results:
CustomerID: 199, Lookalikes: [(71, 0.8343379578499192), (95, 0.8401322968842639), (5, 0.8163254404882451)]
CustomerID: 146, Lookalikes: [(78, 0.691350565709974), (177, 0.8767964371989058), (172, 0.7452405885539201)]
CustomerID: 127, Lookalikes: [(164, 0.6477758346964917), (172, 0.7878108422860945), (90, 0.7412511387354124)]
CustomerID: 87, Lookalikes: [(51, 0.8016865016741538), (102, 0.7882076813423698), (28, 0.8130576812455536)]
CustomerID: 70, Lookalikes: [(174, 0.6775918209040295), (193, 0.6853859778283921), (171, 0.720753004643409)]
CustomerID: 188, Lookalikes: [(165, 0.9069834884751673), (141, 0.8997766543144162), (82, 0.903144445310678)]
CustomerID: 195, Lookalikes: [(12, 0.8566734133555302), (39, 0.9710270106088785), (124, 0.8165543290181443)]
CustomerID: 8, Lookalikes: [(24, 0.8123908168231618), (109, 0.7245765364322511), (194, 0.8212523091133167)]
CustomerID: 157, Lookalikes: [(116, 0.772569054364363), (94, 0.9016761448297896), (47, 0.7550628659260556)]
Cus

In [36]:
# Prepare the data for CSV
lookalike_data = []
# Loop through the first 20 customers (C0001 to C0020)
for customer_id in range(0,21):
    lookalikes = lookalike_dict.get(customer_id, [])
    for lookalike in lookalikes:
        lookalike_data.append([customer_id, lookalike[0], lookalike[1]])



In [37]:
# Create a DataFrame and save to CSV
df_lookalikes = pd.DataFrame(lookalike_data, columns=['CustomerID', 'LookalikeID_Cosine', 'SimilarityScore_Cosine'])
df_lookalikes["CustomerID"] = df_lookalikes["CustomerID"].apply(lambda x: f'C{x:04d}')
df_lookalikes["LookalikeID_Cosine"] = df_lookalikes["LookalikeID_Cosine"].apply(lambda x: f'C{x:04d}')
# Save the DataFrame to CSV
df_lookalikes.to_csv('Lookalike_Customers_Cosine.csv', index=False)

In [38]:
df_lookalikes.head()

,CustomerID,LookalikeID_Cosine,SimilarityScore_Cosine
0,C0001,C0023,0.778458
1,C0001,C0120,0.728469
2,C0001,C0069,0.772285
3,C0002,C0134,0.835651
4,C0002,C0159,0.881361


In [47]:
# Add the scaled data back to the dataframe
scaled_df = pd.DataFrame(scaled_features, columns=features)
scaled_df['CustomerID'] = cust

In [48]:
scaled_df.shape

(199, 13)

In [49]:
scaled_df

,AccountAge,TotalSpend,TransactionCount,AvgSpend,Frequency,AvgQuantity,LastPurchaseRecency,RegionAvgSpend,Books,Clothing,Electronics,Home Decor,CustomerID
0,0.700146,-0.813993,-0.467494,-0.812176,-0.548521,-0.501206,-0.156585,0.519157,-0.955802,-0.899788,-0.326599,0.789789,199
1,-1.269419,-0.490416,-0.467494,-0.189098,0.526224,-0.947444,-0.919700,0.175931,0.045629,-0.899788,-0.181081,0.083888,146
2,-0.799304,-0.128242,0.444578,-0.626814,0.161301,-1.244935,0.352158,0.519157,-0.472112,-0.899788,0.577704,0.657867,127
3,-0.820813,1.715974,0.900614,1.078130,0.346165,1.092499,0.436948,0.813589,1.419841,0.080355,2.481373,-0.955679,87
4,1.508251,-0.186987,-0.467494,0.395184,-0.608385,0.837506,0.111918,0.519157,1.245165,-0.899788,-0.006676,-0.955679,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,-0.593437,-1.679902,-1.835602,-1.227111,-0.657864,-2.732393,3.630724,-1.776099,-0.955802,-0.472710,-0.944670,-0.955679,58
195,0.420535,-1.194182,-1.379566,-0.191479,-0.651910,-0.054969,0.210840,0.813589,-0.955802,-0.899788,-0.911228,0.622426,95
196,-1.512158,-1.774198,-1.835602,-1.953417,-0.046838,2.622455,-0.990359,0.813589,-0.955802,-0.899788,-0.944670,-0.673410,151
197,-1.484504,-1.352135,-1.835602,1.297473,-0.139309,-0.947444,-0.778383,0.175931,0.029652,-0.899788,-0.944670,-0.955679,78


In [50]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

# Function to find top 3 similar customers
def find_similar_customers(customer_id, df, features, top_n=3):
    # Get the target customer data
    target_data = df.loc[df['CustomerID'] == customer_id, features]
    
    # Check if target_data is empty
    if target_data.empty:
        print(f"Customer {customer_id} not found in the dataframe.")
        return []

    target_data = target_data.values.flatten()
    
    # Calculate Euclidean distance to all other customers
    distances = []
    for _, row in df.iterrows():
        if row['CustomerID'] != customer_id:
            other_data = row[features].values.flatten()
            
            # Check if other_data is empty
            if other_data.size == 0:
                print(f"Error: Data for CustomerID {row['CustomerID']} is empty.")
                continue
                
            dist = euclidean_distance(target_data, other_data)
            distances.append((row['CustomerID'], dist))
    
    # Sort by distance (ascending) and take top N similar customers
    distances.sort(key=lambda x: x[1])
    return distances[:top_n]

In [51]:
lookalikes = {}
for customer_id in customer_data['CustomerID']:
    similar_customers = find_similar_customers(customer_id, scaled_df, features, top_n=3)
    lookalikes[customer_id] = similar_customers

# Print the top 3 similar customers and their Euclidean distances
for customer_id, similar_customers in lookalikes.items():
    print(f"CustomerID: {customer_id}, Lookalikes: {similar_customers}")

CustomerID: 199, Lookalikes: [(5.0, 1.3788948998210233), (164.0, 1.4393506195507413), (86.0, 1.4555477020851204)]
CustomerID: 146, Lookalikes: [(177.0, 1.106863644753769), (31.0, 1.8917950401080255), (85.0, 1.9386587369323687)]
CustomerID: 127, Lookalikes: [(90.0, 1.505147355073483), (164.0, 1.8478541462463223), (177.0, 2.08148438455575)]
CustomerID: 87, Lookalikes: [(28.0, 2.4217015998215503), (51.0, 2.4558909179971335), (102.0, 2.6020490153237983)]
CustomerID: 70, Lookalikes: [(174.0, 2.1567888539823525), (179.0, 2.260591574800681), (74.0, 2.3416609560234907)]
CustomerID: 188, Lookalikes: [(165.0, 1.9767815836756848), (99.0, 2.1161141843988864), (82.0, 2.2053268274421374)]
CustomerID: 195, Lookalikes: [(39.0, 0.8139480282909115), (124.0, 1.773269379543787), (48.0, 1.8168678321705727)]
CustomerID: 8, Lookalikes: [(194.0, 2.1240855971769137), (24.0, 2.192901557198856), (116.0, 2.7657292127477353)]
CustomerID: 157, Lookalikes: [(94.0, 1.389941548150993), (116.0, 2.0785983646388924), (18

In [52]:
lookalike_dat_eu = []
for customer_id in range(1,21):
    # Get the top 3 similar customers for each customer_id
    similar_customers = find_similar_customers(customer_id, scaled_df, features, top_n=3)
    
    # Add the result to lookalike_dat_eu
    for lookalike in similar_customers:
        lookalike_dat_eu.append([customer_id, lookalike[0], lookalike[1]])

# Create a DataFrame with the lookalike data
df_lookalikes_eucledian = pd.DataFrame(lookalike_dat_eu, columns=['CustomerID', 'LookalikeID_Eucledian', 'EuclideanDistance'])
df_lookalikes_eucledian["CustomerID"] = df_lookalikes_eucledian["CustomerID"].apply(lambda x: f'C{x:04d}')
df_lookalikes_eucledian["LookalikeID_Eucledian"] = df_lookalikes_eucledian["LookalikeID_Eucledian"].apply(lambda x: f'C{int(x):04d}')


In [53]:
df_lookalikes_eucledian.head()

,CustomerID,LookalikeID_Eucledian,EuclideanDistance
0,C0001,C0069,1.964190
1,C0001,C0120,2.211886
2,C0001,C0181,2.217318
3,C0002,C0159,1.266496
4,C0002,C0106,1.363020


In [54]:
df_lookalikes_eucledian.to_csv('Lookalike_Customers_Eucledian.csv', index=False)

## END